In [ ]:
%matplotlib inline

# import packages
import sys
import os
import time
import datetime
from qgis import processing
import qgis.core
from qgis.core import QgsApplication, QgsProject, QgsCoordinateReferenceSystem, QgsRasterLayer,QgsGeometry,QgsPoint, QgsPointXY
from qgis.analysis import QgsNativeAlgorithms
from osgeo import gdal
import scipy
from scipy import ndimage
from PIL import Image
import numpy as np
import glob
import xarray as xr
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

# this paramter is set to avoid issues with qgis opening GUIs
os.environ["QT_QPA_PLATFORM"] = "offscreen"

The first major step is to create a .txt file of meteorological inputs compatible with the SOLWEIG model.

In [ ]:
# import model data from RD-DMOFTSCIHT (web address here)
ds = xr.open_dataset('/scratch/bweeding/AGU_dataset/CSIRO-BOM-ACCESS1-0_1990_2005.nc')

# create a dataframe with SOLWEIG's required columns, and a length based on the timestamps of the model data
met_df = pd.DataFrame(index = pd.date_range(start = pd.to_datetime(ds.timestamp[0].values), end = pd.to_datetime(ds.timestamp[-1].values), freq = '60min'),columns=['iy','id','it','imin','qn','qh','qe','qs','qf','U','RH','Tair','pres','rain','kdown','snow','ldown','fcld','wuh','xsmd','lai','kdiff','kdir','wdir'])

# fill all columns with umep default value
met_df = met_df.fillna(-999.00)

# fills the time variables in the dataframe based on the timestamp index
met_df['iy'] = np.array(met_df.index.year)
met_df['id'] = np.array(met_df.index.dayofyear)
met_df['it'] = np.array(met_df.index.hour)
met_df['imin'] = np.array(met_df.index.minute)

# insert data into dataframe, rounding to 2 decimal places as needed for umep
met_df['Tair'] = ds.Tair.values.astype(float).round(2)
met_df['RH'] = ds.RH.values.astype(float).round(2)
met_df['U'] = ds.Uwind.values.astype(float).round(2)
met_df['kdown'] = ds.kdown.values.astype(float).round(2)

# save the file in the SOLWEIG compatible format
met_df.to_csv('/mnt/bweeding_workspace/output/projection_metfiles/csiro_hist.txt',index=False,sep=' ')

In [ ]:
# initiate qgis
QgsApplication.setPrefixPath("/usr/lib/python3/dist-packages/qgis", True)
qgs = QgsApplication([], False)
qgs.initQgis()
QgsProject.instance().setCrs(QgsCoordinateReferenceSystem('EPSG:28355'))

sys.path.append(r'/usr/share/qgis/python/plugins')

# activate plugins
from processing.core.Processing import ProcessingConfig

# import and activate the QGIS native processing algorithms - seems to return false but work!?
from processing.core.Processing import Processing
Processing.initialize()
QgsApplication.processingRegistry().addProvider(QgsNativeAlgorithms())

# import UMEP processing tools
from processing_umep_old.processing_umep_provider import ProcessingUMEPProvider
umep_provider = ProcessingUMEPProvider()
QgsApplication.processingRegistry().addProvider(umep_provider)

# import Fusion processing tools
import processing_fusion.fusionUtils as fusionUtils

from processing_fusion.processing_fusion_provider import ProcessingFUSIONProvider

fusion_provider = ProcessingFUSIONProvider()
QgsApplication.processingRegistry().addProvider(fusion_provider)
Processing.initialize()
ProcessingConfig.setSettingValue(fusionUtils.FUSION_DIRECTORY, '/home/bweeding/.wine/drive_c/FUSION_wine/')

In [ ]:

# sets base folder location
base_folder = '/scratch/bweeding/'

# creates run folder
os.mkdir(base_folder+'SOLWEIG_run_csiro_hist_'+datetime.datetime.now().strftime('%d-%m-%Y_%H%M'))

# sets run folder location
run_folder = base_folder+'SOLWEIG_run_csiro_hist_'+datetime.datetime.now().strftime('%d-%m-%Y_%H%M')+'/'

# creates output folder
os.mkdir(run_folder+'SOLWEIG_output_csiro_hist_'+datetime.datetime.now().strftime('%d-%m-%Y_%H%M'))

# sets output folder location
output_folder = run_folder+'SOLWEIG_output_csiro_hist_'+datetime.datetime.now().strftime('%d-%m-%Y_%H%M')+'/'

# creates folder for grid split vectors
os.mkdir(run_folder+'grid_split')

# sets the LAS (lidar) path and filename (this is pubicly available from https://maps.thelist.tas.gov.au/listmap/app/list/map)
las_loc = '/mnt/bweeding_workspace/input/solweig_inputs/franklin_merge.las'

# sets the building shape file path and filename (this is pubicly available from https://maps.thelist.tas.gov.au/listmap/app/list/map)
build_loc = '/mnt/bweeding_workspace/input/solweig_inputs/list_2d_building_polys_hobart.shp'

In [ ]:
#%% Digital elevation model production

# creates a DEM using fusion:gridsurfacecreate in .dtm format
alg_params_DEM = {
    'ADVANCED_MODIFIERS': '',
    'CELLSIZE': 1,
    'CLASS': '2',
    'INPUT': las_loc.replace('/','\\'),
    'MEDIAN': '',
    'MINIMUM': '',
    'SLOPE': '',
    'SMOOTH': '',
    'SPIKE': '',
    'VERSION64': True,
    'XYUNITS': 0,
    'ZUNITS': 0,
    'OUTPUT_DTM': (run_folder+'DEM.dtm').replace('/','\\')
}
    
dem = processing.run('fusion:gridsurfacecreate', alg_params_DEM)

processing.run("fusion:dtm2ascii", {'INPUT':(run_folder+'DEM.dtm').replace('/','\\'),'CSV':False,'RASTER':False,'MULTIPLIER':None,'OUTPUT':(run_folder+'DEM.asc').replace('/','\\')})

